In [ ]:
from operator import index
from dataDownloader  import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_nyse import DB_NYSE
from db_stock import DB_Stock
from portfolio import Portfolio
from assetAllocation import AssetAllocation
from matplotlib.dates import relativedelta
from datetime import datetime, timedelta
from commonHelper import EDateType
from scipy.stats.mstats import winsorize

import datetime
import dataDownloader
import yfinance as yf;
import pandas as pd
import commonHelper
import numpy as np
import os
import warnings
import yfinance as yf
import mplfinance as mpf
import plotly.graph_objects as go

start_date = '2014-12-01'
end_date = '2025-08-10'



symbols = []
with DB_FinancialStatement() as fs:
    symbols = fs.get_symbol_list_with_filter(2022)
    # symbols = symbols[:40]
    # symbols = ['DTST', 'DXLG', 'AEHR', 'AMTX', 'BGI', 'EP', 'PMTS', 'DTST', 'MPU', 'DXLG', 'GWAV', 'ODV']

df_symbols = AssetAllocation.get_stock_data_with_ma(
    symbols=symbols, 
    start_date=start_date, 
    end_date=end_date, 
    mas=[10], 
    type='ma_month',
    use_db_stock=True)

df_symbols = AssetAllocation.filter_close_last_month(df_symbols)

df_hedge = AssetAllocation.get_stock_data_with_ma(
    # symbols=['SPY', '^VIX'], 
    symbols = ['SPY'],
    start_date=start_date, 
    end_date=end_date, 
    mas=[10], 
    type='ma_month',
    use_db_stock=False)

df_hedge = AssetAllocation.filter_close_last_month(df_hedge)

df_alter = AssetAllocation.get_stock_data_with_ma(
    symbols=['GLD'], 
    start_date=start_date, 
    end_date=end_date, 
    mas=[10], 
    type='ma_month',
    use_db_stock=False)


df_alter = AssetAllocation.filter_close_last_month(df_alter)
# df_alter={}
# df_alter['Cash'] = None


# 3-12 상승, 2년 zscore사용
def strategy_power_momentum(df_symbols:dict, df_hedge:dict, df_alter:dict, m:int=3, window:int=24):
    # m = 3
    col_m_return = f"R_{m}_12M" # 중기 모멘텀
    col_m_reversal = f"R_{m}M"  # 단기 모멘텀
    col_z_score = f"R_{m}M_Z"   # Z스코어
    col_over_return = "Can Over Return"

    df_symbols = df_symbols.copy()

    for key, df in df_symbols.items():
        df = df.copy()

        # 필요 없는 컬럼 제거
        df = df[[col for col in df.columns if col not in ['Dividends', 'Adj Close', 'Condition', 'Close']]]

        # 수익률 컬럼 (월별)
        df["Monthly_Return"] = df["Result Close"].pct_change(fill_method=None)

        # R_2_12M, R_2M 계산
        df[col_m_return] = (df["Result Close"].shift(m) / df["Result Close"].shift(12)) - 1
        df[col_m_reversal] = (df["Result Close"] / df["Result Close"].shift(m)) - 1    

        # window = 24  # 36개월 (3년)

        # rolling 평균과 표준편차
        rolling_mean = df[col_m_reversal].rolling(window=window, min_periods=window).mean()
        rolling_std = df[col_m_reversal].rolling(window=window, min_periods=window).std()

        # 각 row별 "과거 36개월 기준 Z-score"
        df[col_z_score] = (df[col_m_reversal] - rolling_mean) / rolling_std

        # 상승횟수 vs 하락횟수 (3~12개월 전 구간)
        def count_up_down(idx):
            # 3~12개월 전 수익률 가져오기
            window = df.loc[idx-12:idx-m, "Monthly_Return"]
            if window.empty:
                return np.nan
            ups = (window > 0).sum()
            downs = (window < 0).sum()
            return 1 if ups > downs else 0

        # 새로운 컬럼 추가
        df[col_over_return] = [count_up_down(i) for i in range(len(df))]
        df = df.dropna(subset=[col_m_return, col_z_score])
        df_symbols[key] = df

    

    df = AssetAllocation.merge_to_dfs(df_symbols, [col_m_reversal, 'Monthly_Return', 'MMA_10'])    
    df = df.loc[df.groupby(df["Date"].dt.to_period("M"))["Date"].idxmax()]
    df = df.sort_values("Date").reset_index(drop=True)

    init_balance = 10000

    df['End Balance'] = None        # 지난 달 Restart Asset의 결과        
    df['Restart Asset'] = None      # 현 월 마지막에 리벨런싱 처리할 대상
    df['Restart Balance'] = None    # 리벨런싱의 자산 배분
    df['Alter Asset'] = None
    df['Alter Balance'] = None
    df['Balance'] = None            # 토탈 비용
    df['Enter Ratio'] = 0.0
    df['VIX'] = 0.0
   

    def get_restart_assets(symbols, df):
        m_return = dict(zip(symbols, df.at[i, col_m_return])) # 중기모멘텀
        zscore = dict(zip(symbols, df.at[i, col_z_score])) # z스코어
        over_return = dict(zip(symbols, df.at[i, col_over_return]))

        symbols = [sym for sym in symbols
                if -2<=zscore[sym]<=2 and  # Z스코어는 상승/하락 모두 포함하는게 좋음.
                    over_return[sym] == 1 and 
                    m_return[sym] > 0]
        
        symbols = sorted(symbols, key=lambda sym: m_return[sym], reverse=True)
        symbols = symbols[:20]
        return symbols

    # 이전달에 투자한 자산의 결과 리스트 반환
    def get_end_balance(prev_assets, s_symbol, s_close, e_symbol, e_close):
        end_balance = []
        if prev_assets:
            start_close_dict = dict(zip(s_symbol, s_close))
            end_close_dict = dict(zip(e_symbol, e_close))

            try:
                change_ratio = [
                    end_close_dict[sym] / start_close_dict[sym]
                    if start_close_dict[sym] != 0 or sym in end_close_dict 
                        else 0
                    for sym in prev_assets
                ]
            except Exception as e:
                raise Exception((f"애러발생! : {prev_assets}, {e}"))

            start_balance = df.at[i-1, 'Restart Balance']
            end_balance = [round(x*y,2) for x,y in zip(start_balance, change_ratio)]

        return end_balance

    # 한번에 자산 배분할때 얼마만큼 배분할 수 있는지
    def get_one_invest_balance(enter_ratio:float, balance:int, count:int):

        balance = enter_ratio*balance
        one_invest_balance = min(balance//10, balance//count)

        return one_invest_balance
     
    # 리스크 관리
    def get_hedge_value(df_hedge, date):   
        enter_ratio = 0
        vix_value = 0

        if '^VIX' in df_hedge:
            df_vix = df_hedge['^VIX']
            mask = df_vix['Date'] == date
            if mask.any():
                vix_value = df_vix.loc[mask, 'Result Close'].iloc[0]

        if vix_value < 15:
            enter_ratio = 1
        elif  15<= vix_value <20:
            enter_ratio = 0.75
        elif 20<= vix_value<30:
            enter_ratio = 0.5
        elif 30<= vix_value:
            enter_ratio = 0.25
        else:
            enter_ratio = 0

        # 단순 이평선 비교
        # if 'SPY' in df_hedge: 
        #     df_spy = df_hedge['SPY']
        #     is_over_spy = (df_spy.loc[df_spy['Date'] == date, 'Result Close'] > 
        #                     df_spy.loc[df_spy['Date'] == date, 'MMA_10']).any()
        #     if not is_over_spy:
        #         enter_ratio = 0

        # 변동성 타깃팅 방식
        if 'SPY' in df_hedge:
            df_spy = df_hedge['SPY']
            if 'Monthly_Return' not in df_spy.columns:
                df_spy['Monthly_Return'] = df_spy['Result Close'].pct_change(fill_method=None)

            # 실제 변동성 (3개월로 측정. 12제곱근 곱함) (만약 20일선으로 하면 252제곱근을 곱해서 처리)
            if 'Vol_3M_Monthly' not in df_spy.columns:
                df_spy['Vol_3M_Monthly'] = df_spy['Monthly_Return'].rolling(window=3).std()
                # Annualized volatility proxy
                df_spy['Vol_3M_Monthly'] = df_spy['Vol_3M_Monthly'] * np.sqrt(12)

            # 목표 변동성
            target_val = 0.10

            vol = df_spy.loc[df_spy['Date'] == date, 'Vol_3M_Monthly'].iloc[0]
            vol_ratio = target_val / vol if vol > 0 else 0
            vol_ratio = np.clip(vol_ratio, 0, 1)

            enter_ratio *= vol_ratio   # 기존 enter_ratio에 곱하기

            # 최근 월 수익률과 3개월 누적
            spy_ret = df_spy.loc[df_spy['Date'] <= date, 'Monthly_Return']
            if len(spy_ret) >= 3:
                # 룩어헤드 방지: 현재 월이 충격 → 다음 월에 적용
                # 따라서 여기선 date 기준으로는 직전 달의 shock을 체크해야 함
                spy_ret_prev = spy_ret.iloc[-2]  # 직전 달
                last_3m_prev = spy_ret.iloc[-4:-1].sum() if len(spy_ret) >= 4 else 0

                if (spy_ret_prev <= -0.04) or (last_3m_prev <= -0.06):
                    enter_ratio *= 0.6   # 감축
            
        return (enter_ratio, vix_value)

    # 대체 자산 전환
    def get_alter_balance(df_alter, date, alter_dict):
        result = []
        for key, value in alter_dict.items():
            if key == 'GLD':
                df_gld = df_alter['GLD']
                if 'Monthly_Return' not in df_gld.columns:
                    df_gld['Monthly_Return'] = df_gld['Result Close'].pct_change(fill_method=None)
                
                mask = df_gld['Date'] == date
                ratio = 1 + df_gld.loc[mask, 'Monthly_Return'].iloc[0]
                result.append(round(value * ratio))

            if key == 'Cash':
                result.append(value)

        return result

    def get_dd_ratio(dd_prev):
        if dd_prev <= -0.15:
            m_dd = 0.4
        elif dd_prev <= -0.10:
            m_dd = 0.6
        elif dd_prev <= -0.05:
            m_dd = 0.8
        else:
            m_dd = 1.0
        return m_dd


    alter_len = len(df_alter.keys())

    peek_balance = init_balance

    for i in range(len(df)):
        date = df.at[i, 'Date']
        
        curr_assets = get_restart_assets(df.at[i, 'Symbol'], df)

        df.at[i,'Restart Asset'] = curr_assets
        df.at[i,'Alter Asset'] = list(df_alter.keys())

        enter_ratio, vix_value = get_hedge_value(df_hedge, date)

        if vix_value == 0:
            df.at[i, 'VIX'] = int(vix_value)
        
        df.at[i, 'Enter Ratio'] = enter_ratio
        
        if i == 0:
            if not curr_assets:
                df.at[0, 'Alter Balance'] = [init_balance//alter_len] * alter_len
            else:
                curr_assets_len = len(curr_assets)
                one_invest_balance = get_one_invest_balance(enter_ratio, init_balance, curr_assets_len)

                df.at[0,'Restart Balance'] = [one_invest_balance]*curr_assets_len
                leftover_balance = init_balance - (one_invest_balance*curr_assets_len)
                df.at[0, 'Alter Balance'] = [leftover_balance] * alter_len
                
            df.at[0, 'Balance'] = init_balance

        else:
            prev_assets = df.at[i-1, 'Restart Asset']

            end_balance = get_end_balance(prev_assets, 
                                        df.at[i-1,'Symbol'], 
                                        df.at[i-1,'Result Close'],
                                        df.at[i, 'Symbol'],
                                        df.at[i, 'Result Close'])
            
            df.at[i, 'End Balance'] = end_balance

            alter_balance = get_alter_balance(df_alter,
                                              date,
                                              dict(zip(df.at[i-1, 'Alter Asset'], df.at[i-1, 'Alter Balance'])))

            df.at[i, 'Alter Balance'] = alter_balance
            
            now_total_balance = 0
            if end_balance:
                now_total_balance += sum(end_balance)

            if alter_balance:
                now_total_balance += sum(alter_balance)
            
            df.at[i, 'Balance'] = now_total_balance

            peek_balance = max(peek_balance, now_total_balance)

            dd = now_total_balance/ peek_balance - 1 
            df.at[i,'DD'] = dd
            enter_ratio *= get_dd_ratio(dd)

            if not curr_assets:
                df.at[0, 'Alter Balance'] = [now_total_balance//alter_len] * alter_len   
            else:
                curr_assets_len = len(curr_assets)
                one_invest_balance = get_one_invest_balance(enter_ratio, now_total_balance, curr_assets_len)

                restart_balance = [one_invest_balance] * curr_assets_len
                leftover_balance = int(now_total_balance - (one_invest_balance * curr_assets_len))

                df.at[i, 'Restart Balance'] = restart_balance
                df.at[i, 'Alter Balance'] = [leftover_balance] * alter_len
    
    return df


allocations = [
    # {'3R 24 GLD 30' : strategy_power_momentum(df_symbols, df_hedge, df_alter, 3, 24)},
    # {'2R 24 GLD 30' : strategy_power_momentum(df_symbols, df_hedge, df_alter, 2, 24)},
    # {'1R 24 GLD 30' : strategy_power_momentum(df_symbols, df_hedge, df_alter, 1, 24)},
    # {'3R 36 GLD 30' : strategy_power_momentum(df_symbols, df_hedge, df_alter, 3, 36)},
    # {'2R 36 GLD 30' : strategy_power_momentum(df_symbols, df_hedge, df_alter, 2, 36)},
    {'1R 36 SPY DD+Circuit' : strategy_power_momentum(df_symbols, df_hedge, df_alter, 1, 36)}, 
]

# strategy_power_momentum(df_symbols, df_hedge, df_alter, 1, 36)

# portfolio_name = [
#     # '1R 36',
#     # '1R 36 SPY',
#     '1R 36 VIX',
#     # '1R 36 GLD 30',
#     '1R 36 SPY Vol',
#     '1R 36 SPY Vol GLD'
# ]

# allocations = [
#     {'SPY':Portfolio.spy(start_date,end_date)}
#     ]
# for name in portfolio_name:
#     key_value = {}
#     key_value[name] = pd.read_csv(f"stocks/power_momentum_{name}.csv")
#     allocations.append(key_value)

Portfolio.show_portfolio(allocations)
for key_value in allocations:
    for key, value in key_value.items():
        value.to_csv(f"stocks/power_momentum_{key}.csv")

# df = pd.read_csv("stocks/power_momentum_1R 36.csv")
# display(df)

# df = strategy_power_momentum(df_symbols, df_spy, 2, 24)
# display(df)


# df = Portfolio.spy(start_date,end_date)
# df.to_csv('stocks/spy.csv',index=False)
